In [4]:
import sys
sys.path.append('C:\\Users\\monty.minh\\Documents\\Optimizer\\')

from sympy import Matrix

from model.data import *
from model.preprocessing import *
from model.optimization import *
from model.postprocessing import *

In [22]:
class Results:
    volume, cost = [], []

In [26]:
Data.filepath = 'C:/Users/monty.minh/Documents/Optimizer/0. Inputs/Inputs Example.xlsm'
Results.save_location = 'C:/Users/monty.minh/Documents/Optimizer/6. Results/Results.xlsx'

get_timeframe()

for Data.year in range(Data.timeframe[0], Data.timeframe[1]):
    preprocess()
    optimize()
    postprocess()

In [2]:
import pandas as pd

df = pd.DataFrame({'First' : [5, 2, 0, 10, 4], 
                   'Second' : [9, 8, 21, 3, 8]})

writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')

df.to_excel(writer, sheet_name='Sheet1')

workbook  = writer.book
workbook.filename = 'test.xlsm'
workbook.add_vba_project('./vbaProject.bin')

writer.save()

C:\Users\monty.minh\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:300: UserWarning: VBA project binary file './vbaProject.bin' not found.
  warn("VBA project binary file '%s' not found." % vba_project)
